In [1]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
import datasetdatabase as dsdb

os.environ["DSDB_PROCESS_LIMIT"] = "16"

# MEM, DNA and FOV Datasets

Connecting to dsdb

In [2]:
prod = dsdb.DatasetDatabase(config='/allen/aics/assay-dev/Analysis/QCB_database/prod_config.json')

Loading metadata

In [3]:
with open('tmp_meta_drug_ds.pkl', 'rb') as fp:
    df_meta = pickle.load(fp)

df_meta.head()

,cell_id,cell_ver,czi_filename,drug_label,idx_in_stack,roi,str_ver,structure_name
0,0ade9e16-db16-420a-9ccb-8cad3b32ee04,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,1,"[35, 141, 113, 380, 264, 469]",0.1.0,tubulin
1,ca1f5d66-da95-4208-bd89-65c6c614a9f1,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,2,"[35, 139, 183, 427, 327, 608]",0.1.0,tubulin
2,662de948-69c0-4b93-92f8-f3bcc7e38655,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,3,"[35, 139, 225, 458, 23, 406]",0.1.0,tubulin
3,7806500f-6df9-4b9f-8b23-fde6fd09d40e,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,4,"[35, 144, 403, 529, 156, 439]",0.1.0,tubulin
4,17fcb999-72a5-425d-b485-b02edb16651c,0.1.0,3500001171_100X_20170811_1-Scene-1-P7-D05.czi,Paclitaxol,9,"[35, 147, 293, 450, 168, 397]",0.1.0,tubulin


Upload exiting local pkl files

In [5]:
DSToIngest = [
    {'file': 'tmp_mem_drug_ds.pkl',
     'name': 'QCB_DRUG_MEM_feature',
     'description': 'Cell membrane features extracted from QCB DRUG data',
     'na_allowed': False,
     'meta_id': 'cell_id'},
    {'file': 'tmp_dna_drug_ds.pkl',
     'name': 'QCB_DRUG_DNA_feature',
     'description': 'Nucleus features extracted from QCB DRUG data',
     'na_allowed': False,
     'meta_id': 'cell_id'}]

In [6]:
for data in DSToIngest:
    
    print('running:',data['file'])
    
    with open(data['file'], 'rb') as fp:
        df = pickle.load(fp)
        
    if data['na_allowed'] is False:
        assert df.isnull().values.any() == False
    
    if data['meta_id'] is not None:
        df = pd.concat([df.reset_index(drop=True), df_meta[data['meta_id']]], axis=1)
    
    ds = dsdb.Dataset(df, name=data['name'], description=data['description'])
    
    ds.upload_to(prod)

running: tmp_mem_drug_ds.pkl
Storing algorithm parameters.
Input dataset already exists in database. 1
Tearing down object...
[============================================================] 100.0% 1519/1519 ~ 0:0:0 remaining
running: tmp_dna_drug_ds.pkl
Storing algorithm parameters.
Input dataset already exists in database. 1
Tearing down object...
[============================================================] 100.0% 1519/1519 ~ 0:0:0 remaining


# Structures Datasets

Uploading existing structures pkl files

In [11]:
DSToIngest = [
    {'file': 'tmp_str_golgi_drug_ds.pkl',
     'name': 'QCB_DRUG_ST6GAL_feature',
     'description': 'Features extracted from QCB images of ST6GAL-tagged Golgi.',
     'na_allowed': True,
     'meta_id': 'cell_id'},
    {'file': 'tmp_str_er_drug_ds.pkl',
     'name': 'QCB_DRUG_SEC61B_feature',
     'description': 'Features extracted from QCB DRUG images of SEC61B-tagged ER.',
     'na_allowed': True,
     'meta_id': 'cell_id'},
    {'file': 'tmp_str_tubulin_drug_ds.pkl',
     'name': 'QCB_DRUG_TUBA1B_feature',
     'description': 'Features extracted from QCB DRUG images of TUBA1B-tagged microtubules.',
     'na_allowed': True,
     'meta_id': 'cell_id'}]

In [12]:
for data in DSToIngest:
    
    print('running:',data['file'])
    
    with open(data['file'], 'rb') as fp:
        df = pickle.load(fp)
        
    if data['na_allowed'] is False:
        assert df.isnull().values.any() == False
    
    df = df.reset_index(drop=False)
    df = df.rename(index=str, columns={"index":"cell_id"})
    
    ds = dsdb.Dataset(df, name=data['name'], description=data['description'])
    
    ds.upload_to(prod)

running: tmp_str_golgi_drug_ds.pkl
Storing algorithm parameters.
Input dataset already exists in database. 1
Tearing down object...
[============================================================] 100.0% 231/231 ~ 0:0:0 remaining
